# Digit Recognizer Competition Solution

This solution uses KNN, which is implemented in pure Python. I'm well aware of the benefits that CNN's bring in terms of image classification, but this implementation acted more as a test in terms of my understanding of the algorithm itself. The final result on the test data when k = x was %

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
from scipy import stats

## Getting the data

In my opinion, MNIST didn't require a closer look in terms of EDA. From what I've learned so far, it's that sometimes unnecessary data exploration can lead to additional bias towards your data.

In [2]:
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')
    
X = train_data.iloc[:, 1:]
y = train_data.iloc[:, :1]
train_X, validation_X, train_y, validation_y = train_test_split(X.values, y.values, test_size = 0.30, random_state = 1)
    
test_X = test_data.values

In [3]:
print(train_data.shape)
train_data.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating distances between vectors

I decided to use the NearPy's locality-sensitive hashing engine to do the random binary projections, in order to store the vectors, and calculate distances between them efficiently. Later on, I've implemented brute force euclidean distance calculator in case the number of neighbors is less than k when querying the LSH engine.

In [4]:
dimensions = 784

hyper_plane_count = 1
hash_layers = 12
engine = Engine(dimensions, lshashes=[RandomBinaryProjections('rpb', hyper_plane_count) for i in range(hash_layers)])

labels = {}

# Save the train data vectors to the LSH engine
for vector_id in range(len(train_X)):
    labels['data_%d' % vector_id] = train_y[vector_id][0]
    engine.store_vector(train_X[vector_id], 'data_%d' % vector_id)    

In [5]:
def get_K_nearest_neighbours(vector, k):
    '''Brute force approach to get k nearest neighbours'''
    distances = []
    print("Shape:: ", vector.shape[1])
    for vector_id in range(len(train_X)):
        current_vector = train_X[vector_id]
        label = train_y[vector_id][0]
        sum_of_squares = 0
        for i in range(784):
            sum_of_squares += np.square(vector[i] - current_vector[i])
        distance = np.sqrt(sum_of_squares)
        distances.append(([current_vector, label], distance))
        
        if len(distances) > k:
            distances.sort(key = lambda x: x[1])
            distances = distances[:k]
    
    distances.sort(key = lambda x: x[1])
    
    k_closest_neighbours = []
    for i in range(k):
        k_closest_neighbours.append(distances[i][0])

    return k_closest_neighbours

In [6]:
def get_predictions(X, k):
    predictions = []
    for vector in X:
        neighbours = engine.neighbours(vector)
        neighbours.sort(key = lambda x: x[2])
        neighbours = neighbours[:k]
        nearest_labels = list(map(lambda neighbour: labels.get(neighbour[1]), neighbours))
        if len(neighbours) < k:
            neighbours = get_K_nearest_neighbours(vector, k)
            nearest_labels = list(map(lambda neighbour: neighbour[1], neighbours))

        prediction = stats.mode(nearest_labels)

        if len(prediction) > 0: 
            prediction = int(prediction[0])
        else:
            prediction = -1

        predictions.append(prediction)
    
    return predictions

In [7]:
k = 3
validation_predictions = get_predictions(validation_X, k)
accuracy_score(validation_y, validation_predictions)

0.9719047619047619

In [8]:
predictions = get_predictions(test_X, k)
   
my_submission = pd.DataFrame({'ImageId': [i for i in range(1, len(predictions) + 1)], 'Label': predictions})
my_submission.to_csv('submission.csv', index = False)